In [1]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
%config InlineBackend.figure_format ='retina'

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## Import data

In [13]:
#btc options data
df=pd.read_csv('data/btc_data_bsm.csv', sep=",", index_col=0)
df_ex=pd.read_csv('data/btc_data_risk.csv', sep=",", index_col=0)
df = df.reset_index(drop=False)
#df.drop(columns=['Unnamed: 0', 'interest_rate'], inplace=True)

In [14]:
df_ex

,index,best_bid_amount,best_ask_amount,bid_iv,ask_iv,underlying_index,underlying_price,mark_iv,interest_rate,best_bid_price,...,Ending_Net_Asset_Value,Monthly_Rate_of_Return,Unrealized_Gain_Loss,Change_in_Open_Trade_Equity,Net_Performance_wo_Fees,Net_Performance,Additions_Withdrawals,Additions,Withdrawals,VAMI
0,1,0.0,92.5,0.00,110.01,SYN.BTC-4FEB24,43046.6290,85.49,0.0,0.0000,...,1.003546e+06,0.003546,-4547.683580,0.000000,3546.381644,2546.120149,1000.261494,1000.261494,0.0000,1.000000e+06
1,2,0.0,19.8,0.00,139.25,SYN.BTC-4FEB24,43046.8715,78.77,0.0,0.0000,...,1.003158e+06,0.003158,-4047.939779,499.743802,3657.501802,2766.852104,390.905896,390.905896,0.0000,1.003158e+06
2,3,0.0,1.5,0.00,90.44,SYN.BTC-4FEB24,43046.8715,78.77,0.0,0.0000,...,1.003157e+06,0.003157,-4047.939879,-0.000100,3156.562087,2266.249673,890.312514,890.312514,0.0000,1.006324e+06
3,4,0.0,20.7,0.00,126.92,SYN.BTC-4FEB24,43046.8715,66.52,0.0,0.0000,...,1.002768e+06,0.002768,-3547.953577,499.986302,3268.166749,2487.397903,280.782545,280.782545,0.0000,1.009110e+06
4,5,0.0,26.8,0.00,80.04,SYN.BTC-4FEB24,43046.8715,66.52,0.0,0.0000,...,1.002767e+06,0.002767,-3547.953577,0.000000,2766.562697,1986.250139,780.312558,780.312558,0.0000,1.011902e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
652,653,0.0,0.0,0.00,0.00,BTC-27DEC24,46185.0600,64.49,0.0,0.0000,...,9.999987e+05,-0.000001,1.312000,1.282000,-0.030000,-0.030000,-1.282000,0.000000,1.2820,6.524448e+07
653,654,16.0,0.1,63.81,65.54,BTC-27DEC24,46185.0600,64.68,0.0,0.0230,...,1.000883e+06,0.000883,0.022500,-1.289500,882.041602,632.896355,250.434747,250.434747,0.0000,6.530212e+07
654,655,0.0,0.0,0.00,0.00,BTC-27DEC24,46185.0600,64.68,0.0,0.0000,...,9.999984e+05,-0.000002,1.608800,1.586300,-0.022500,-0.022500,-1.586300,0.000000,1.5863,6.530201e+07
655,656,17.0,0.1,63.86,66.23,BTC-27DEC24,46185.2600,65.05,0.0,0.0175,...,1.000693e+06,0.000693,0.018500,-1.590300,691.643590,496.115609,197.118282,197.118282,0.0000,6.534728e+07


In [1]:
import requests
import pandas as pd
from datetime import datetime

# Alpha Vantage API configuration
API_KEY = 'NW05RFO7K02SAMAL'
SYMBOL = 'BTC'
MARKET = 'USD'

def fetch_historical_data(symbol, market, api_key):
    url = f'https://www.alphavantage.co/query?function=DIGITAL_CURRENCY_DAILY&symbol={symbol}&market={market}&apikey={api_key}'
    response = requests.get(url)
    data = response.json()
    time_series = data.get('Time Series (Digital Currency Daily)', {})
    
    # Convert to DataFrame
    df = pd.DataFrame.from_dict(time_series, orient='index')
    df.index = pd.to_datetime(df.index)
    df = df.sort_index()
    
    # Rename columns for convenience
    df.rename(columns={
        '1a. open (USD)': 'open',
        '2a. high (USD)': 'high',
        '3a. low (USD)': 'low',
        '4a. close (USD)': 'close',
        '5. volume': 'volume',
        '6. market cap (USD)': 'market_cap'
    }, inplace=True)
    
    # Convert to numeric
    df = df.apply(pd.to_numeric)
    return df

# Fetch the data
historical_data = fetch_historical_data(SYMBOL, MARKET, API_KEY)
print(historical_data.head())


             1. open   2. high    3. low  4. close        volume
2023-07-20  29916.69  30421.29  29564.19  29809.13  11494.953488
2023-07-21  29809.13  30060.28  29729.48  29907.98   7597.810267
2023-07-22  29907.15  30002.38  29626.86  29795.03   3629.482511
2023-07-23  29793.62  30350.70  29733.55  30081.61   4513.832143
2023-07-24  30081.61  30099.99  28850.00  29176.98  14484.275440
